In [2]:
import email
import imaplib
from io import BytesIO

import pandas as pd

In [8]:
prev = 0


def pretty(label: str, completed: float, total: int, length: int = 30) -> None:
    global prev
    print(
        " " * (prev * 2)
        + f"\r{label} ["
        + "=" * int(completed / total * length)
        + "-" * int((total - completed) / total * length)
        + f"] {int(completed / total * 100)}%",
        end="",
    )
    prev = length


def prettyPrintReplace(label: str) -> None:
    global prev
    print(" " * (prev * 2) + f"\r{label}", end="")
    prev = len(label)

In [4]:
# Login credentials
IMAP_HOST = "imap.gmail.com"
EMAIL_USER = "ebadansari414@gmail.com"
EMAIL_PASS = "ghgy lcsp obph sczi"

In [5]:
subject_to_search = "STATE CODE - 27 | "

In [6]:
# Connect and login
mail = imaplib.IMAP4_SSL(IMAP_HOST)
mail.login(EMAIL_USER, EMAIL_PASS)
mail.select("inbox")

# Search emails by subject
status, messages = mail.search(None, f'(SUBJECT "{subject_to_search}")')

In [ ]:
rawGRN = pd.DataFrame()
rawPR = pd.DataFrame()
totalMailParsed = 0
if status == "OK":
    for num in messages[0].split():
        typ, data = mail.fetch(num, "(RFC822)")
        msg = email.message_from_bytes(data[0][1])

        # Extract only selected attachments
        for part in msg.walk():
            content_disposition = str(part.get("Content-Disposition") or "")
            prettyPrintReplace(
                f"Mail parsed: {totalMailParsed}, GRN: {len(rawGRN)}, PR: {len(rawPR)}"
            )
            if part.get_content_maintype() == "multipart":
                continue
            if "attachment" in content_disposition:
                filename = part.get_filename()
                # if filename and filename.lower() in [f.lower() for f in allowed_files]:
                if filename.lower().endswith("25.csv") and filename.lower().startswith(
                    "grn"
                ):
                    file_data = part.get_payload(decode=True)
                    file_stream = BytesIO(file_data)
                    rawGRN = pd.concat([rawGRN, pd.read_csv(file_stream)])
                elif filename.lower().endswith(
                    "25.csv"
                ) and filename.lower().startswith("pr"):
                    file_data = part.get_payload(decode=True)
                    file_stream = BytesIO(file_data)
                    rawPR = pd.concat([rawPR, pd.read_csv(file_stream)])
        totalMailParsed += 1

Mail parsed: 19, GRN: 68, PR: 68                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [6]:
mail.logout()

('BYE', [b'LOGOUT Requested'])

In [ ]:
print(breakThisHere())

NameError: name 'breakThisHere' is not defined

In [ ]:
grn = rawGRN.dropna(how="all", axis=1)
grn.to_csv("GRN.csv")

In [ ]:
pr = rawPR.dropna(how="all", axis=1)
pr.to_csv("PR.csv")